# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)

In [ ]:
import numpy as np
import operator
import nltk
import pandas as pd

!pip install conll-df
from conll_df import conll_df

**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.



In [ ]:
# !git clone https://github.com/UniversalDependencies/UD_English-GUM

In [ ]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM
%cd /content/UD_English-GUM/
!git checkout 2c8b062269f2d2d3d62405c82d8c25cf24f705dd
%cd ..

fatal: destination path 'UD_English-GUM' already exists and is not an empty directory.
/content/UD_English-GUM
HEAD is now at 2c8b062 Updated statistics.
/content


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

In [ ]:
train_data = conll_df('UD_English-GUM/en_gum-ud-train.conllu', file_index=False, skip_morph=True)[['w','x']]
dev_data = conll_df('UD_English-GUM/en_gum-ud-dev.conllu', file_index=False, skip_morph=True)[['w','x']]
test_data = conll_df('UD_English-GUM/en_gum-ud-test.conllu', file_index=False,skip_morph=True)[['w','x']]

In [ ]:
train_sentences = train_data.reset_index().groupby('s').agg(','.join) #for HMM

In [ ]:
train_sentences

,w,x
s,,
1,"Aesthetic,Appreciation,and,Spanish,Art,:","ADJ,NOUN,CCONJ,ADJ,NOUN,PUNCT"
2,"Insights,from,Eye-Tracking","NOUN,ADP,NOUN"
3,"Claire,Bailey-Ross,claire.bailey-ross@port.ac....","PROPN,PROPN,PROPN,PROPN,ADP,PROPN,PUNCT,PROPN,..."
4,"Andrew,Beresford,a.m.beresford@durham.ac.uk,Du...","PROPN,PROPN,PROPN,PROPN,PROPN,PUNCT,PROPN,PROPN"
5,"Daniel,Smith,daniel.smith2@durham.ac.uk,Durham...","PROPN,PROPN,PROPN,PROPN,PROPN,PUNCT,PROPN,PROPN"
...,...,...
4283,"Spread,out,quinoa,evenly,and,cover,baking,dish...","VERB,ADP,NOUN,ADV,CCONJ,VERB,NOUN,NOUN,ADV,ADP..."
4284,"Cook,the,quinoa,in,the,oven,for,roughly,20,min...","VERB,DET,NOUN,ADP,DET,NOUN,ADP,ADV,NUM,NOUN,PU..."
4285,"Remove,the,aluminum,foil,from,the,baking,dish,...","VERB,DET,NOUN,NOUN,ADP,DET,NOUN,NOUN,PUNCT,VER..."


In [ ]:
train_data = train_data.reset_index()

In [ ]:
train_data

,s,i,w,x
0,1,1,Aesthetic,ADJ
1,1,2,Appreciation,NOUN
2,1,3,and,CCONJ
3,1,4,Spanish,ADJ
4,1,5,Art,NOUN
...,...,...,...,...
81856,4286,11,.,PUNCT
81857,4287,1,Serve,VERB
81858,4287,2,and,CCONJ
81859,4287,3,enjoy,VERB


In [ ]:
test_data = test_data.reset_index()
dev_data = dev_data.reset_index()

In [ ]:
test_data

,s,i,w,x
0,1,1,The,DET
1,1,2,prevalence,NOUN
2,1,3,of,ADP
3,1,4,discrimination,NOUN
4,1,5,across,ADP
...,...,...,...,...
15921,890,14,with,ADP
15922,890,15,your,PRON
15923,890,16,nesting,VERB
15924,890,17,box,NOUN


In [ ]:
xValues = test_data.reset_index().groupby('x').agg(','.join)

In [ ]:
xValues

,w
x,
ADJ,"racial,contemporary,representative,Personal,mu..."
ADP,"of,across,in,from,of,of,of,of,of,across,of,In,..."
ADV,"nationally,already,also,Indeed,Even,so,At,howe..."
AUX,"have,been,have,have,have,have,should,be,is,are..."
CCONJ,"and,and,and,and,and,and,and,or,or,or,and,or,an..."
DET,"The,a,the,the,the,the,the,the,a,the,the,the,th..."
INTJ,"Yes,please"
NOUN,"prevalence,discrimination,groups,Results,sampl..."
NUM,"1,3,3,5,6,7,9,10,11,12,13,14,one,15,33,4,one,1..."


In [ ]:
xValues = xValues.reset_index()['x'].values

In [ ]:
xValues

['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', ..., 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
Length: 17
Categories (17, object): ['ADJ', 'ADP', 'ADV', 'AUX', ..., 'SCONJ', 'SYM', 'VERB', 'X']

**Part 2**

Write a class **simple_tagger**, with methods *train* and *evaluate*. The method *train* receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method *evaluate* receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.


In [ ]:
def updateDict (key, dictionary):
  if key in dictionary:
    dictionary[key] += 1
  else:
    dictionary[key] = 1

In [ ]:
class simple_tagger:
  def train(self, data):
    #occs_dict: keys are tuples (word,tag), values are their number of occurences 
    occs_dict = {}
    # print(len(data.index))
    for i,row in data.iterrows():
      updateDict((row['w'], row['x']), occs_dict)
  
    #max_dict: keys are the words and values are the current max of occurences
    max_dict = {}
    self.final_dict = {}
    for tup in occs_dict:
      if tup[0] in max_dict:
        if occs_dict[tup] > max_dict[tup[0]]:
          max_dict.update({tup[0] : occs_dict[tup]})
          self.final_dict.update({tup[0] : tup[1]})
      else:
          max_dict.update({tup[0] : occs_dict[tup]})
          self.final_dict.update({tup[0] : tup[1]})

  def evaluate(self, data):
    totalWords = 0
    correctWords = 0
    totalSentences = 0
    correctSentences = 0
    prevS = 1
    allCorrect = True

    #checks what is the tags' mode (for OOVs' assignments)
    max = 0
    most_freq_tag = ''
    for tag in xValues:
      cur = sum(map((tag).__eq__, self.final_dict.values()))
      if cur > max:
        max = cur
        most_freq_tag = tag
      
    for i,row in data.iterrows():
      curS = row['s'] #current sentence number
      if curS != prevS:
        totalSentences += 1
        if allCorrect:
          correctSentences += 1
        allCorrect = True
    
      totalWords += 1
      if row['w'] in self.final_dict:
        if row['x'] == self.final_dict[row['w']]: #if the tag is correct
          correctWords += 1
        else:
          allCorrect = False
      else:
        if row['x'] == most_freq_tag:
          correctWords += 1
        else:
          allCorrect = False
      prevS = curS
    print("correct words: " + str(correctWords))
    print("total words: " + str(totalWords))
    print("correct sentences: " + str(correctSentences))
    print("total sentences: " + str(totalSentences))
    return (correctWords/totalWords), (correctSentences/totalSentences)
        


In [ ]:
s_t = simple_tagger()
s_t.train(data = train_data)
simple_test_word_acc, simple_test_sen_acc  = s_t.evaluate(test_data)
simple_dev_word_acc, simple_dev_sen_acc  = s_t.evaluate(dev_data)
print(simple_test_word_acc, simple_test_sen_acc)

correct words: 13475
total words: 15926
correct sentences: 169
total sentences: 889
correct words: 13363
total words: 15598
correct sentences: 124
total sentences: 783
0.8461007158106242 0.19010123734533182


**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.


In [ ]:
def update_for_A(tag1, tag2, diction):
  if tag1 not in diction: 
    d = {tag1 : {tag2 : 1}}
    diction.update(d)
  else:
    if tag2 not in diction[tag1]:
      d1 = {tag2 : 1}
      diction[tag1].update(d1)
    else:
      diction[tag1][tag2] = diction[tag1][tag2] + 1
  


In [ ]:
def update_for_B(tag1, word, diction):
  if tag1 not in diction:
    d = {tag1 : {word : 1}}
    diction.update(d)
  else:
    if word not in diction[tag1]:
      d1 = {word : 1}
      diction[tag1].update(d1)
    else:
      diction[tag1][word] = diction[tag1][word] + 1



In [ ]:
def id_to_tag_fun(data):
  id_to_tag_dict = {}
  tag_to_id_dict = {}
  j = 0
  for i,row in train_data.iterrows():
     curTag = row['x']
     if (curTag not in tag_to_id_dict):
       d = {j : curTag}
       c = {curTag : j}
       tag_to_id_dict.update(c)
       id_to_tag_dict.update(d)
       j += 1
  return id_to_tag_dict, tag_to_id_dict

In [ ]:
def word_to_id_fun(data):
  word_to_id = {}
  id_to_word = {}
  j = 0
  for i,row in train_data.iterrows():
    if (row['w'] not in word_to_id):
      c = {j : row['w']}
      d = {row['w'] : j}
      word_to_id.update(d)
      id_to_word.update(c)
      j += 1
  return word_to_id, id_to_word

In [ ]:
class hmm_tagger:
  def __init__(self):
    self.A = None
    self.B = None
    self.pi = None
    self.id_to_tag_dict = None
    self.tag_to_id_dict = None
    self.word_to_id_dict = None
    self.id_to_word_dict = None
    self.Pi_dict = None
    self.A_dict = None
    self.B_dict = None


  def train(self, data):
    word_to_id_dict, id_to_word_dict = word_to_id_fun(data)
    self.word_to_id_dict = word_to_id_dict
    self.id_to_word_dict = id_to_word_dict
    occsPi = {}
    prevSentence = 0

    #### Pi ####
    for i,row in data.iterrows(): 
      curTag = row['x']
      curSentence = row['s']
      if (curSentence != prevSentence): #take only first words in each sentence and consider their tags
        updateDict(curTag, occsPi)
      prevSentence = curSentence


    id_to_tag_dict, tag_to_id_dict = id_to_tag_fun(data)
    self.id_to_tag_dict = id_to_tag_dict
    self.tag_to_id_dict = tag_to_id_dict
    self.Pi_dict = occsPi
    pi_matrix = np.zeros(len(id_to_tag_dict))
    for i in range(len(id_to_tag_dict)):
      pi_matrix[i] = occsPi[id_to_tag_dict[i]]

    pi_matrix = pi_matrix / np.sum(pi_matrix)

    #### A ####
    occsA = {}
    prevTag = '/s'
    for i,row in data.iterrows(): #A

      curSentence = row['s']
      curTag = row['x']

      if curSentence != prevSentence: #deals with the addition of the special symbol '\s'
        update_for_A('/s', curTag, occsA)
        update_for_A(prevTag, '/s', occsA)
      else:
        update_for_A(prevTag, curTag, occsA)
      prevTag = row['x']
      prevSentence = curSentence

    self.A_dict = occsA
    A_matrix = np.zeros((len(self.id_to_tag_dict),len(self.id_to_tag_dict)))
    for i in range(A_matrix.shape[0]):
      for j in range(A_matrix.shape[1]):
        tag1 = self.id_to_tag_dict[i]
        tag2 = self.id_to_tag_dict[j]
        if tag1 in occsA:
          if tag2 in occsA[tag1]:
            A_matrix[i][j] = occsA[tag1][tag2]
          else:
            A_matrix[i][j] = 0 # no transition
        else:
          A_matrix[i][j] = 0 # no transition

    A_matrix = A_matrix / np.sum(A_matrix, axis=1)[:,None] # devide each row by it's sum (to get probability)

    #### B ####
    dictonaryB = {}
    prevTag = '/s'
    for i,row in data.iterrows():
      curWord = row['w']
      curTag = row['x']
      update_for_B(curTag, curWord, dictonaryB)

    self.B_dict = dictonaryB
    B_matrix = np.zeros((len(dictonaryB),len(word_to_id_dict)))
    for i in range(B_matrix.shape[0]):
      for j in range(B_matrix.shape[1]):
        tag = self.id_to_tag_dict[i]
        word = self.id_to_word_dict[j]
        if tag in dictonaryB:
          if word in dictonaryB[tag]:
            B_matrix[i][j] = dictonaryB[tag][word]
          else:
            B_matrix[i][j] = 0
        else:
          B_matrix[i][j] = 0

    B_matrix = B_matrix / np.sum(B_matrix, axis=1)[:,None] # devide each row by it's sum (to get probability)
    
  
    self.A = A_matrix
    self.B = B_matrix
    self.pi = pi_matrix



  def evaluate(self, data):

    def checkSentence(A,B): # checks whether all sentences in A were corretly classified 
      for i in range(len(A)):
        if A[i] != B[i]:
          return 0
      return 1

    #### Convert observations into the list of IDs #####
    sequance_ids = []
    known = 0
    for index, row in data.iterrows():
      if (row['w'] in self.word_to_id_dict):

        sequance_ids.append(self.word_to_id_dict[row['w']])
        known = known + 1
      else:
        sequance_ids.append(-1)

    #### Collecting gold-standard tags of observation ####
    ground_truth_tags = []
    for index, row in data.iterrows():
        ground_truth_tags.append(row['x'])

    temp = []
    temp_truth = []
    final_arr = []
    final_arr_truth = []
    for i in sequance_ids:

      if (i != -1):
        temp.append(i)
      else:
        if (temp != []):
          final_arr.append(temp)
        temp = [-1]
        final_arr.append(temp)
        temp = []
    if (temp != []):
      final_arr.append(temp)
    

    #### Run Viterbi ####
    result = []
    for row in final_arr:
      seq = viterbi(row, self.A, self.B, self.pi, self.word_to_id_dict, self.id_to_word_dict, self.id_to_tag_dict)
      result.append(seq)

    flatten_list_result = sum(result, [])

    #### Check accuracy on words ####
    correct_predicted = 0
    for i in range(len(flatten_list_result)):
      if (flatten_list_result[i] == ground_truth_tags[i]):
        correct_predicted = correct_predicted + 1

    
    accuracy_word = correct_predicted / len(flatten_list_result)
    print("Word accuracy: ", accuracy_word)

    #### Check accuracy on sentences ####
    i = 0
    prevSen = 1
    arrayA = []
    arrayB = []
    correct = 0
    total = 0
    for index, row in data.iterrows():
      if (row['s'] == prevSen):
        arrayA.append(flatten_list_result[i])
        arrayB.append(ground_truth_tags[i])
        prevSen = row['s']
        i = i + 1
      
      else:
        correct = correct + checkSentence(arrayA, arrayB)
        total = total + 1
        arrayA = []
        arrayB = []
        arrayA.append(flatten_list_result[i])
        arrayB.append(ground_truth_tags[i])
        prevSen = row['s']
        i = i + 1
      
    accuracy_sen = correct / total
    print("Sentence accuracy: ", accuracy_sen)
    return accuracy_word, accuracy_sen



    




In [ ]:
def viterbi (observations, A, B, Pi, word_to_id_dict, id_to_word_dict, id_to_tag_dict):
  #### OOV word ####
  if (observations == [-1]):
    randomTag = id_to_tag_dict[np.random.randint(0, B.shape[0])]
    return [randomTag]
  
  score_matrix = np.zeros((B.shape[0],len(observations)))
  psi_matrix = np.zeros((B.shape[0],len(observations)))
  

  
  #### Initialization Step ####
  first_obs_id = observations[0] # extracting first ovservation-word id
  for i in range(score_matrix.shape[0]):
    score_matrix[i][0] = np.log(B[i][first_obs_id]) + np.log2(Pi[i])
    psi_matrix[i][0] = 0


  #### Iteration Step ####
  j = 0
  for row in observations: # going over observation (cols)
    if (j == 0): 
      j = j + 1
      continue #skip first observation 
    for i in range(score_matrix.shape[0]): # going over states (rows)
      prev_col = score_matrix[:,j-1]
      transitions = np.log(A[:,i]) #j column of A (transition probabilities from all states to state i)
      emission = np.log(B[i][row])
      score_matrix[i][j] = emission + np.amax(np.add(prev_col, transitions))
      psi_matrix[i][j] = np.argmax(np.add(prev_col, transitions))
    j = j + 1

  #### Sequence recovery ####
  Sequance_ids = []
  tag_id = np.argmax(score_matrix[:, -1])

  for j in range(psi_matrix.shape[1]-1, -1, -1):
    Sequance_ids.insert(0,tag_id)
    tag_id = psi_matrix[int(tag_id)][j] # get the prev tag
  
  Sequance = []
  for tag_id in Sequance_ids:
    Sequance.append(id_to_tag_dict[tag_id])
  
  return Sequance


In [ ]:
h_m = hmm_tagger()
h_m.train(data = train_data)


In [ ]:
hmm_test_word_acc, hmm_test_sen_acc = h_m.evaluate(data = test_data)
hmm_dev_word_acc, hmm_dev_sen_acc = h_m.evaluate(data = dev_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log


Word accuracy:  0.8077357779731257
Sentence accuracy:  0.13048368953880765
Word accuracy:  0.8305551993845365
Sentence accuracy:  0.12643678160919541


**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over both, the dev and test datasets. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [ ]:
#### Convert dataframe to the list of sentences ####
def data_format(data):
  prevSen = 1
  sentence = []
  list_of_sentences = []
  for index, row in data.iterrows():
    if (row['s'] == prevSen):
      sentence.append((row['w'],row['x']))
      prevSen = row['s']
    else:
      list_of_sentences.append(sentence)
      sentence = []
      prevSen = row['s']
    
  return list_of_sentences

      
    
      

In [ ]:
print(data_format(train_data))

[[('Aesthetic', 'ADJ'), ('Appreciation', 'NOUN'), ('and', 'CCONJ'), ('Spanish', 'ADJ'), ('Art', 'NOUN'), (':', 'PUNCT')], [('from', 'ADP'), ('Eye-Tracking', 'NOUN')], [('Bailey-Ross', 'PROPN'), ('claire.bailey-ross@port.ac.uk', 'PROPN'), ('University', 'PROPN'), ('of', 'ADP'), ('Portsmouth', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('Beresford', 'PROPN'), ('a.m.beresford@durham.ac.uk', 'PROPN'), ('Durham', 'PROPN'), ('University', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('Smith', 'PROPN'), ('daniel.smith2@durham.ac.uk', 'PROPN'), ('Durham', 'PROPN'), ('University', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('Warwick', 'PROPN'), ('c.l.h.warwick@durham.ac.uk', 'PROPN'), ('Durham', 'PROPN'), ('University', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('do', 'AUX'), ('people', 'NOUN'), ('look', 'VERB'), ('at', 'ADP'), ('and', 'CCONJ'), ('experience', 'VERB'), ('art', 'NOUN'),

In [ ]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(data_format(train_data))
tnt_test_word_acc = tnt_pos_tagger.evaluate(data_format(test_data))
tnt_dev_word_acc = tnt_pos_tagger.evaluate(data_format(dev_data))

In [ ]:
#### Sentence level accuracy ####
def tnt_sentence_level(data):
  correct = 0
  total = 0
  dt = data_format(data)
  for sentence in dt:
    if sentence != []:
      ev = tnt_pos_tagger.evaluate([sentence])
      if (ev == 1):
        correct = correct + 1
    total = total + 1
  acc = correct/total
  return acc

In [ ]:
tnt_test_sen_acc = tnt_sentence_level(test_data)
tnt_dev_sen_acc = tnt_sentence_level(dev_data)

Print both, word level and sentence level accuracy for all the three taggers in a table.

In [ ]:
from tabulate import tabulate
data = [["simple_tagger", 'dev_set', simple_dev_word_acc, simple_dev_sen_acc],
["simple_tagger", 'test_set', simple_test_word_acc, simple_test_sen_acc],
["hmm_tagger", 'dev_set', hmm_dev_word_acc, hmm_dev_sen_acc],
["hmm_tagger", 'test_set', hmm_test_word_acc, hmm_test_sen_acc],
["MEMM_tagger", 'dev_set', tnt_dev_word_acc, tnt_dev_sen_acc],
["MEMM_tagger", 'test_set', tnt_test_word_acc, tnt_test_sen_acc]
] 
print (tabulate(data, headers=["Tagger", "Set", "Word level", "Sentence level"]))


Tagger         Set         Word level    Sentence level
-------------  --------  ------------  ----------------
simple_tagger  dev_set       0.856712          0.158365
simple_tagger  test_set      0.846101          0.190101
hmm_tagger     dev_set       0.830555          0.126437
hmm_tagger     test_set      0.807736          0.130484
MEMM_tagger    dev_set       0.830508          0.140485
MEMM_tagger    test_set      0.806791          0.152981
